In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *
import pickle

class Rnnlm:
    def __init__(self, vocab_size = 10000, wordvec_size = 100, hidden_size = 100):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_W = (rn(V,D)/100).astype('f')
        lstm_Wx = (rn(D, 4*H)/np.sqrt(D)).astype('f')
        lstm_Wh = (rn(D, 4*H)/np.sqrt(D)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')
        affine_W = (rn(H,V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        self.layers = [
            TimeEmbedding(embed_W),
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful = True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.lstm_layer = self.layers[1]
        
        self.params, self.grads = [],[]
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads
            
    def predict(self, xs):
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs
    
    def forward(self, xs, ts):
        score = self.predict(xs)
        loss = self.loss_layer.forward(score, ts)
        return loss
    
    def backward(self, dout = 1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self):
        self.lstm_layer.reset_state()
    
    def save_params(self, file_name = 'Rnnlm.pkl'):
        with open(file_name, 'wb') as f:
            pickle.dump(self.params, f)
    
    def load_params(self, file_name = 'Rnnlm.pkl'):
        with open(file_name, 'rb') as f:
            self.params = pickle.load(f)

In [2]:
import sys
sys.path.append('..')
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity
from dataset import ptb
# from rnnlm import Rnnlm

batch_size = 20
wordvec_size = 100
hidden_size = 100
time_size = 35
lr = 20.0
max_epoch = 4
max_grad = 0.25

corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_test, _, _ = ptb.load_data('test')
vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = Rnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

trainer.fit(xs, ts, max_epoch, batch_size, time_size, max_grad, eval_interval = 20)
trainer.plot(ylim=(0,500))

model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print("테스트 퍼플렉서티: ", ppl_test)

model.save_params()

Done
| 에폭 1 |  반복 1 / 1327 | 시간 0[s] | 퍼플렉서티 10001.78
| 에폭 1 |  반복 21 / 1327 | 시간 6[s] | 퍼플렉서티 3017.74
| 에폭 1 |  반복 41 / 1327 | 시간 13[s] | 퍼플렉서티 1217.64
| 에폭 1 |  반복 61 / 1327 | 시간 20[s] | 퍼플렉서티 953.77
| 에폭 1 |  반복 81 / 1327 | 시간 28[s] | 퍼플렉서티 799.95
| 에폭 1 |  반복 101 / 1327 | 시간 36[s] | 퍼플렉서티 657.39
| 에폭 1 |  반복 121 / 1327 | 시간 43[s] | 퍼플렉서티 634.41
| 에폭 1 |  반복 141 / 1327 | 시간 51[s] | 퍼플렉서티 612.55
| 에폭 1 |  반복 161 / 1327 | 시간 59[s] | 퍼플렉서티 568.94
| 에폭 1 |  반복 181 / 1327 | 시간 66[s] | 퍼플렉서티 559.90
| 에폭 1 |  반복 201 / 1327 | 시간 74[s] | 퍼플렉서티 487.53
| 에폭 1 |  반복 221 / 1327 | 시간 82[s] | 퍼플렉서티 490.55
| 에폭 1 |  반복 241 / 1327 | 시간 89[s] | 퍼플렉서티 439.90
| 에폭 1 |  반복 261 / 1327 | 시간 97[s] | 퍼플렉서티 468.78
| 에폭 1 |  반복 281 / 1327 | 시간 104[s] | 퍼플렉서티 440.02
| 에폭 1 |  반복 301 / 1327 | 시간 112[s] | 퍼플렉서티 385.22
| 에폭 1 |  반복 321 / 1327 | 시간 120[s] | 퍼플렉서티 341.04
| 에폭 1 |  반복 341 / 1327 | 시간 127[s] | 퍼플렉서티 395.53
| 에폭 1 |  반복 361 / 1327 | 시간 135[s] | 퍼플렉서티 404.52
| 에폭 1 |  반복 381 / 1327 | 시간 143[s] | 퍼플렉서티 

<Figure size 640x480 with 1 Axes>

퍼플렉서티 평가 중 ...
234 / 235
테스트 퍼플렉서티:  134.50922962524314
